In [24]:
from sklearn import linear_model
import numpy as np
from collections import namedtuple
tokenized_row = namedtuple('tokenized_row', 'sent_count sentences word_count words')
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import csv


def train_sgd(train_targets, train_regressors):
    sgd = linear_model.SGDClassifier()
    sgd.fit(train_regressors, train_targets)
    return sgd

def error_rate(train_targets, train_regressors, test_targets, test_regressors):
    sgd = train_sgd(train_targets, train_regressors)
    test_predictions = sgd.predict(test_regressors)
    rounded_predictions = np.rint(test_predictions)
    false_pos = 0
    false_neg = 0
    correct = 0
    for i in range(len(rounded_predictions)):
        if rounded_predictions[i] == 1 and test_targets[i] == 0: false_pos += 1
        if rounded_predictions[i] == 0 and test_targets[i] == 1: false_neg += 1
        if rounded_predictions[i] == test_targets[i]: correct += 1
    errors = false_pos + false_neg
    corrects = len(rounded_predictions) - errors
    assert(correct == corrects)
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, false_pos, false_neg)

filenames = ['combined_train_test.p', 'r_train_so_test.p', 'so_train_r_test.p',
            'so_alone.p', 'reddit_alone.p']


In [25]:
def baseline(filename):
    with open(filename, 'rb') as pfile:
        train, test = pickle.load(pfile)
    train_targets = train['answer_good'].values.reshape(-1, 1)
    train_regressors = train['AnswerCount'].values.reshape(-1, 1)
    test_targets = test['answer_good'].values.reshape(-1, 1)
    test_regressors = test['AnswerCount'].values.reshape(-1, 1)
    return error_rate(train_targets, train_regressors, test_targets, test_regressors)

with open('baseline_results.csv', 'w+', newline="") as csvfile:
    fieldnames = ['Test Name', 'Success Rate', 'false +', 'false -']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for name in filenames:
        errors, false_pos, false_neg = baseline(name)
        success_rate = 1 - errors
        writer.writerow({'Test Name': name, 'Success Rate': success_rate, 
                         'false +': false_pos, 'false -': false_neg})
        

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
def length_only(filename):
    with open(filename, 'rb') as pfile:
        train, test = pickle.load(pfile)
    # Get length from the dict! Word count and sentence count
    directory_name = filename.split('.p')[0]
    with open(directory_name + "/tokenized_dict.p", 'rb') as pfile:
        train_token_dict, test_token_dict = pickle.load(pfile)
    train_length = len(train.index.values)
    train_regressors = np.empty([train_length, 4])
    test_length = len(test.index.values)
    test_regressors = np.empty([test_length, 4])
    for i in range(train_length):
        index = train.index.values[i]
        row = train_token_dict[index]
        train_regressors[i] = [row[0].word_count, row[0].sent_count, row[1].word_count, row[1].sent_count]
    for i in range(test_length):
        index = test.index.values[i]
        row = test_token_dict[index]
        test_regressors[i] = [row[0].word_count, row[0].sent_count, row[1].word_count, row[1].sent_count]
    test_targets = test['answer_good'].values.reshape(-1, 1)
    train_targets = train['answer_good'].values.reshape(-1, 1)
    return error_rate(train_targets, train_regressors, test_targets, test_regressors)

with open('length_only_results.csv', 'w+', newline="") as csvfile:
    fieldnames = ['Test Name', 'Success Rate', 'false +', 'false -']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for name in filenames:
        errors, false_pos, false_neg = length_only(name)
        success_rate = 1 - errors
        writer.writerow({'Test Name': name, 'Success Rate': success_rate, 
                         'false +': false_pos, 'false -': false_neg})

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
def words_and_length(filename):
    with open(filename, 'rb') as pfile:
        train, test = pickle.load(pfile)
    # Get length from the dict! Word count and sentence count
    train_regressors = []
    test_regressors = []
    directory_name = filename.split('.p')[0]
    with open(directory_name + "/tokenized_dict.p", 'rb') as pfile:
        train_token_dict, test_token_dict = pickle.load(pfile)
    with open(directory_name + "/body_vectorizer.p", 'rb') as pfile:
        body_vectorizer = pickle.load(pfile) 
    with open(directory_name + "/title_vectorizer.p", 'rb') as pfile:
        title_vectorizer = pickle.load(pfile)   
    for i in train.index.values:
        row = train_token_dict[i]
        title_words = row[0].words
        body_words = row[1].words
        title_vectorization = title_vectorizer.transform(title_words)
        body_vectorization = body_vectorizer.transform(body_words)
        # Append the vectorization of the word input
        train_regressors.append((title_vectorization, body_vectorization, 
                                 row[0].word_count, row[0].sent_count, row[1].word_count, row[1].sent_count))
    train_targets = train['answer_good'].values
    test_targets = test['answer_good'].values
    for i in test.index.values:
        row = test_token_dict[i]
        title_words = row[0].words
        body_words = row[1].words
        title_vectorization = title_vectorizer.transform(title_words)
        body_vectorization = body_vectorizer.transform(body_words)
        # Append the vectorization of the word input
        test_regressors.append((title_vectorization, body_vectorization,
                               row[0].word_count, row[0].sent_count, row[1].word_count, row[1].sent_count))
    return error_rate(train_targets, train_regressors, test_targets, test_regressors)

with open('words_and_length_results.csv', 'w+', newline="") as csvfile:
    fieldnames = ['Test Name', 'Success Rate', 'false +', 'false -']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for name in filenames:
        errors, false_pos, false_neg = words_and_length(name)
        success_rate = 1 - errors
        writer.writerow({'Test Name': name, 'Success Rate': success_rate, 
                         'false +': false_pos, 'false -': false_neg})
